### Create agentsClient to connect to AI foundry project

In [11]:
import os
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

agents_client = AgentsClient(
    #endpoint=os.environ["PROJECT_ENDPOINT"],
    endpoint = 'eastus.api.azureml.ms;95717284-5de1-45e3-a461-5fe0db848584;aoairg;azure-ai-projects-for-agents',
    credential=DefaultAzureCredential(),
)

In [12]:
import os
import asyncio
from azure.ai.agents.aio import AgentsClient
from azure.core.credentials import AzureKeyCredential

agent_client = AgentsClient(
   #endpoint=os.environ["PROJECT_ENDPOINT"],
   endpoint = 'eastus.api.azureml.ms;95717284-5de1-45e3-a461-5fe0db848584;aoairg;azure-ai-projects-for-agents',
   credential=DefaultAzureCredential(),
)

In [ ]:
import os
from pathlib import Path
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool
from dotenv import load_dotenv
load_dotenv('agents.env')

# Create an AIProjectClient from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

code_interpreter = CodeInterpreterTool()
with project_client:
    # Create an agent with the Code Interpreter tool
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You politely help with math questions. Use the Code Interpreter tool when asked to visualize numbers.",  # Instructions for the agent
        tools=code_interpreter.definitions,  # Attach the tool
    )
    print(f"Created agent, ID: {agent.id}")

    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content="Hi, Agent! Draw a graph for a line with a slope of 4 and y-intercept of 9.",  # Message content
    )
    print(f"Created message, ID: {message['id']}")

    # Create and process an agent run
    run = project_client.agents.runs.create_and_process(
        thread_id=thread.id,
        agent_id=agent.id,
        additional_instructions="Please address the user as Jane Doe. The user has a premium account",
    )
    print(f"Run finished with status: {run.status}")

    # Check if the run failed
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Fetch and log all messages
    messages = project_client.agents.messages.list(thread_id=thread.id)
    for message in messages:
        print(f"Role: {message.role}, Content: {message.content}")

        # Save every image file in the message
        for img in message.image_contents:
            file_id = img.image_file.file_id
            file_name = f"{file_id}_image_file.png"
            project_client.agents.files.save(file_id=file_id, file_name=file_name)
            print(f"Saved image file to: {Path.cwd() / file_name}")

    # Uncomment these lines to delete the agent when done
    # project_client.agents.delete_agent(agent.id)
    # print("Deleted agent")

Created agent, ID: asst_CFpjrcMHhYQ08TC9yGcMaWRT
Created thread, ID: thread_nbBlNZUwrTv7fEv4yzANICst
Created message, ID: msg_E9pxrplnZzlgSVIU4GYvpwAz
Run finished with status: RunStatus.COMPLETED
Role: MessageRole.AGENT, Content: [{'type': 'image_file', 'image_file': {'file_id': 'assistant-SxypNmnL8ozAy5ndALFtcX'}}, {'type': 'text', 'text': {'value': 'Here is the graph of a line with a slope of 4 and a y-intercept of 9. The line passes through the point (0, 9) and has a slope of 4. Let me know if you need any further assistance!', 'annotations': []}}]


ResourceNotFoundError: (None) No such File object: assistant-SxypNmnL8ozAy5ndALFtcX
Code: None
Message: No such File object: assistant-SxypNmnL8ozAy5ndALFtcX

In [9]:
for message in messages:
    img = message.image_contents  # Display images in the notebook

In [10]:
img

{'type': 'image_file', 'image_file': {'file_id': 'assistant-QUNpkrUZf2QsKTvqhVwHuU'}}

In [18]:
project_client.agents.files.get(file_id = 'assistant-QUNpkrUZf2QsKTvqhVwHuU') # Display the first image in the notebook

ValueError: HTTP transport has already been closed. You may check if you're calling a function outside of the `with` of your client creation, or if you called `close()` on your client already.